In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_201904_dataprotection_by_design_and_by_default_v2.0_en.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/protection.md"

In [3]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [43]:
import re
import pandas as pd
file_path = "../original/protection.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [44]:
columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
#columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
subpoint_pattern = re.compile(r'^(\d\.\d\.\d\.\d)\s+(.+)$') 

annex_pattern = re.compile(r'(?i)^ANNEX -- (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        subpoint_match = subpoint_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                subpoint = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, subpoint, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            subpoint = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, subpoint, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            subpoint = ""
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, subpoint, heading, text, section_reference])
        elif subpoint_match:
            match = subpoint_match
            section = section
            subsection = subsection
            point = point
            subpoint = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = subpoint
            data.append([section, subsection, point, subpoint, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, subpoint, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4]) # boilerplate 
df.reset_index(drop=True, inplace=True)

df.loc[df["section_reference"] == "",  "section_reference"] = "Summary"
df.at[0, "heading"] = True
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [41]:
import pandas as pd
from IPython.display import display

# Increase the display width
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Display the DataFrame slice
display(df[220:240])

,section,subsection,point,subpoint,heading,text,section_reference
220,,,,,False,- Protection according to risk - All categorie...,3.8
221,,,,,False,- Security incident response management - Have...,3.8
222,,,,,False,- Incident management - Controller should have...,3.8
223,,,,,False,Example <br> A controller wants to extract lar...,3.8
224,3,3.9,,,True,"Accountability[^41] \n[^41]: See Recital 74, w...",3.9
225,,,,,False,86. The principle of accountability states tha...,3.9
226,,,,,False,87. The controller needs to be able to demonst...,3.9
227,,,,,False,88. To be able to process personal data respon...,3.9
228,4,,,,True,ARTICLE 25(3) CERTIFICATION,4
229,,,,,False,"89. According to Article 25(3), certification ...",4


In [45]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 8 augmented with footnotes
Row 18 augmented with footnotes
Row 19 augmented with footnotes
Row 22 augmented with footnotes
Row 26 augmented with footnotes
Row 28 augmented with footnotes
Row 32 augmented with footnotes
Row 44 augmented with footnotes
Row 48 augmented with footnotes
Row 65 augmented with footnotes
Row 76 augmented with footnotes
Row 80 augmented with footnotes
Row 86 augmented with footnotes
Row 88 augmented with footnotes
Row 93 augmented with footnotes
Row 105 augmented with footnotes
Row 110 augmented with footnotes
Row 111 augmented with footnotes
Row 114 augmented with footnotes
Row 116 augmented with footnotes
Row 120 augmented with footnotes
Row 122 augmented with footnotes
Row 139 augmented with footnotes
Row 140 augmented with footnotes
Row 143 augmented with footnotes
Row 144 augmented with footnotes
Row 156 augmented with footnotes
Row 157 augmented with footnotes
Row 175 augmented with footnotes
Row 182 augmented with footnotes
Row 191 augmented with foo

In [46]:
file = "../inputs/documents/protection.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [48]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.protection
importlib.reload(gdpr_rag.documents.protection)
from gdpr_rag.documents.protection import Protection

path_to_manual_as_csv_file = "../inputs/documents/protection.parquet"
doc = Protection(path_to_manual_as_csv_file)


In [51]:
from IPython.display import Markdown, display

section = "1"
section = "2.2.1"
#section = "Summary"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


2 ANALYSIS OF ARTICLE 25(1) AND (2) DATA PROTECTION BY DESIGN AND BY DEFAULT
2.2 Article 25(2): Data protection by default
2.2.1 By default, only personal data which are necessary for each specific purpose of the processing are processed


# 2 ANALYSIS OF ARTICLE 25(1) AND (2) DATA PROTECTION BY DESIGN AND BY DEFAULT

## 2.2 Article 25(2): Data protection by default

### 2.2.1 By default, only personal data which are necessary for each specific purpose of the processing are processed

40. A "default", as commonly defined in computer science, refers to the pre-existing or preselected value of a configurable setting that is assigned to a software application, computer program or device. Such settings are also called "presets" or "factory presets", especially for electronic devices.

41. Hence, the term "by default" when processing personal data, refers to making choices regarding configuration values or processing options that are set or prescribed in a processing system, such as a software application, service or device, or a manual processing procedure that affect the amount of personal data collected, the extent of their processing, the period of their storage and their accessibility.

42. The controller should choose and be accountable for implementing default processing settings and options in a way that only processing that is strictly necessary to achieve the set, lawful purpose is carried out by default. Here, controllers should rely on their assessment of the necessity of the processing with regards to the legal grounds of Article 6(1). This means that by default, the controller shall not collect more data than is necessary, they shall not process the data collected more than is necessary for their purposes, nor shall they store the data for longer than necessary. The basic requirement is that data protection is built into the processing by default.

43. The controller is required to predetermine for which specified, explicit and legitimate purposes the personal data is collected and processed.[^13] The measures must by default be appropriate to ensure that only personal data which are necessary for each specific purpose of processing are being processed. The EDPS "Guidelines to assess necessity and proportionality of measures that limit the right to data protection of personal data" can be useful also to decide which data is necessary to process in order to achieve a specific purpose.[^14],[^15],[^16]

44. If the controller uses third party software or off-the-shelf software, the controller should carry out a risk assessment of the product and make sure that functions that do not have a legal basis or are not compatible with the intended purposes of processing are switched off.

45. The same considerations apply to organisational measures supporting processing operations. They should be designed to process, at the outset, only the minimum amount of personal data necessary for the specific operations. This should be particularly considered when allocating data access to staff with different roles and different access needs.

46. Appropriate "technical and organisational measures" in the context of data protection by default is thus understood in the same way as discussed above in subchapter 2.1.1, but applied specifically to implementing the principle of data minimisation.

47. The aforementioned obligation to only process personal data which are necessary for each specific purpose applies to the following elements.

  
[^13]: Art. 5(1)(b), (c), (d), (e) GDPR.  
[^14]: EDPS. "Guidelines on assessing the necessity and proportionality of measures that limit the right to data protection". 25 February 2019. edps.europa.eu/sites/edp/files/publication/19-02- 25_proportionality_guidelines_en.pdf  
[^15]: See also EDPS. "Assessing the necessity of measures that limit the fundamental right to the protection of personal data: A Toolkit" https://edps.europa.eu/data-protection/our-work/publications/papers/necessity- toolkit_en  
[^16]: For more information on necessity, see Article 29 Working Party. "Opinion 06/2014 on the notion of legitimate interests of the data controller under Article 7 of Directive 95/46/EC". WP 217, 9 April 2014. ec.europa.eu/justice/article-29/documentation/opinion-recommendation/files/2014/wp217_en.pdf